In [1]:
import pandas as pd
import numpy as np
from mido import MidiFile
import IPython
import matplotlib.pyplot as plt
import librosa.display
import keras.layers as L
import keras.models as M
import keras
from keras.layers import SimpleRNN,LSTM,GRU
from tensorflow.keras.utils import to_categorical

import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Input, Concatenate, Dropout, LSTM, Conv2DTranspose, Conv2D, LeakyReLU, GlobalMaxPooling2D, Reshape, Flatten
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Embedding
from tensorflow.keras.losses import CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import RMSprop, Adam

from sklearn.model_selection import train_test_split
from IPython import *
import os
import tensorflow as tf

from numpy.random import choice

from mido import Message, MidiFile, MidiTrack

2023-06-11 03:00:09.106573: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-11 03:00:14.997003: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/llvm-10/lib:/glob/development-tools/versions/oneapi/2023.1.2/oneapi/tbb/2021.9.0/env/../lib/intel64/gcc4.8:/glob/development-tools/versions/oneapi/2023.1.2/oneapi/rkcommon/1.11.0/lib:/glob/development-tools/versions/oneapi/2023.1.2/oneapi/ospray_studio/0.12.0/lib:/glob/development-tools/versions/oneapi/2023.1.2/oneapi/ospray/2.11.0/lib:/glob/development-tools/versions/oneapi/20

In [2]:
key_notes = {
    'Cb': 59,
    'C': 60,
    'C#': 61,
    'Db': 61,
    'D': 62,
    'D#': 63,
    'Eb': 63,
    'E': 64,
    'F': 65,
    'F#': 66,
    'Gb': 66,
    'G': 67,
    'G#': 68,
    'Ab': 68,
    'A': 69,
    'A#': 70,
    'Bb': 70,
    'B': 71
}

In [3]:
data=pd.read_csv('musicnet-dataset/musicnet_metadata.csv')

In [4]:
data.head()

,id,composer,composition,movement,ensemble,source,transcriber,catalog_name,seconds
0,1727,Schubert,Piano Quintet in A major,2. Andante,Piano Quintet,European Archive,http://tirolmusic.blogspot.com/,OP114,447
1,1728,Schubert,Piano Quintet in A major,3. Scherzo: Presto,Piano Quintet,European Archive,http://tirolmusic.blogspot.com/,OP114,251
2,1729,Schubert,Piano Quintet in A major,4. Andantino - Allegretto,Piano Quintet,European Archive,http://tirolmusic.blogspot.com/,OP114,444
3,1730,Schubert,Piano Quintet in A major,5. Allegro giusto,Piano Quintet,European Archive,http://tirolmusic.blogspot.com/,OP114,368
4,1733,Schubert,Piano Sonata in A major,2. Andantino,Solo Piano,Museopen,Segundo G. Yogore,D959,546


In [5]:
mid=MidiFile('musicnet-dataset/musicnet_midis/musicnet_midis/Beethoven/2313_qt15_1.mid',clip=True)
mid.tracks

[<midi track 'L.v.Beethoven 1mov - String Quartet No.15 in A minor Op.132\x00' 325 messages>,
 <midi track 'Violin1' 3277 messages>,
 <midi track 'Violin2' 2898 messages>,
 <midi track 'Viola' 2916 messages>,
 <midi track 'Cello' 2491 messages>,
 <midi track 'Violin1 Sub' 40 messages>]

In [6]:
for i in mid.tracks[1] :
    if 'meta' in str(i):
        print(i)

<meta message track_name name='Violin1' time=0>
<meta message key_signature key='Am' time=0>
<meta message key_signature key='Dm' time=52>
<meta message key_signature key='Am' time=128>
<meta message key_signature key='Em' time=4>
<meta message key_signature key='Am' time=384>
<meta message end_of_track time=0>


In [7]:
mid.tracks[2]

<midi track 'Violin2' 2898 messages>

In [8]:
k = 0
for i in mid.tracks[2] :
    print(i)
    k += 1
    if k > 50:
        break

<meta message track_name name='Violin2' time=0>
control_change channel=1 control=0 value=0 time=0
program_change channel=1 program=40 time=0
control_change channel=1 control=7 value=100 time=0
control_change channel=1 control=10 value=44 time=0
control_change channel=1 control=91 value=127 time=0
<meta message key_signature key='Am' time=0>
note_on channel=1 note=69 velocity=1 time=0
control_change channel=1 control=11 value=127 time=0
note_off channel=1 note=69 velocity=100 time=188
control_change channel=1 control=11 value=90 time=2092
note_on channel=1 note=59 velocity=40 time=24
control_change channel=1 control=11 value=100 time=24
control_change channel=1 control=11 value=114 time=24
control_change channel=1 control=11 value=127 time=24
note_off channel=1 note=59 velocity=100 time=692
note_on channel=1 note=57 velocity=45 time=4
note_off channel=1 note=57 velocity=100 time=764
note_on channel=1 note=56 velocity=50 time=4
note_off channel=1 note=56 velocity=100 time=764
control_cha

In [9]:
beethoven_midi_traks = {}
n=20
name = None
for m in range(n):
    mid=MidiFile('musicnet-dataset/musicnet_midis/musicnet_midis/Beethoven/'+os.listdir('musicnet-dataset/musicnet_midis/musicnet_midis/Beethoven')[m],clip=True)
    print(mid.tracks)
    for j in range(len(mid.tracks)):
        if j == 0:
            name = mid.tracks[j].name + ': '
            print(name)
        else:
            beethoven_midi_traks[name + mid.tracks[j].name] = mid.tracks[j]

[<midi track 'L.v.Beethoven 1mov - Piano Sonata No.27 in E Minor Op.90\x00' 99 messages>, <midi track 'Right' 2641 messages>, <midi track 'Left' 2648 messages>]
L.v.Beethoven 1mov - Piano Sonata No.27 in E Minor Op.90 : 
[<midi track 'L.v.Beethoven 2mov - Piano Sonata No.30 in E Major Op.109\x00' 48 messages>, <midi track 'Right' 1491 messages>, <midi track 'Left' 1420 messages>]
L.v.Beethoven 2mov - Piano Sonata No.30 in E Major Op.109 : 
[<midi track 'L.v.Beethoven 2mov - Piano Sonata No.18 in Eb Major Op.31-3\x00' 34 messages>, <midi track 'Right' 2883 messages>, <midi track 'Left' 3150 messages>]
L.v.Beethoven 2mov - Piano Sonata No.18 in Eb Major Op.31-3 : 
[<midi track 'Trio für Klarinette, Cello und Klavier Op.38, Allegro con brio' 84 messages>, <midi track 'Staff' 2134 messages>, <midi track 'Staff-1' 2234 messages>, <midi track 'Staff-2' 5710 messages>, <midi track 'Staff-3' 5104 messages>, <midi track 'Staff-4' 8 messages>]
Trio für Klarinette, Cello und Klavier Op.38, Allegr

In [10]:
def get_key(s):
    k = None
    if 'key' in s:
        k = s[33:35]
        if k[-1] == "m" or k[-1] == "'":
            k = k[:-1]
    return k




In [11]:
# function returning list of dicts with each note, it's duration and velocity 
def parse_notes(track):
    key = 'C'
    tunes = []
    new_tune = []
    note_dict = {}
    for i in track:
        
        if i.is_meta:
            new_key = get_key(str(i))
            if new_key is not None:
                key = new_key
            if len(tunes) > 0:
                tunes.append(new_tune)
                new_tune = []
                
        elif i.type == 'note_on' or i.type == 'note_off':
            if i.type == 'note_on' and i.dict()['velocity'] > 0 and i.dict()['time'] > 0:
                note_dict['time'] = i.dict()['time']
                note_dict['note'] = i.dict()['note']
                note_dict['velocity'] = i.dict()['velocity']
                note_dict['channel'] = i.dict()['channel']
            elif i.type == 'note_off' or i.type == 'note_on' and i.dict()['velocity'] == 0:
                if note_dict:
                    note_dict['pause'] = i.dict()['time']
                    note_dict['key'] = key
                    new_tune.append(note_dict)
                    note_dict = {}
    tunes.append(new_tune)
    return tunes

In [12]:
def tune_to_midi(tune, midi_name='new_tune', debug_mode=False):
    mid = MidiFile()
    track = MidiTrack()
    mid.tracks.append(track)
    for note in tune:
        if debug_mode:
            track.append(Message('note_on', note=note, time=64))
            track.append(Message('note_off', note=note, time=128))
        else:
            track.append(Message('note_on', note=note['note'], velocity=note['velocity'], time=note['time']))
            track.append(Message('note_off', note=note['note'], time=note['pause']))

    mid.save(midi_name + '.mid')

In [13]:
tunes = []
max_key = max(key_notes.values())
for k, v in beethoven_midi_traks.items():
    if 'Right' in k:
        new_tunes = parse_notes(v)
        if len(new_tunes) > 0:
            tunes.append(pd.DataFrame(new_tunes[0]))

In [14]:
tunes[2]

,time,note,velocity,channel,pause,key
0,4,61,103,0,188,Ab
1,4,56,80,0,572,Ab
2,4,65,100,0,188,Ab
3,4,61,80,0,188,Ab
4,4,63,80,0,188,Ab
...,...,...,...,...,...,...
1180,196,80,70,0,188,Ab
1181,196,75,70,0,188,Ab
1182,4,75,70,0,188,Ab
1183,4,68,70,0,380,Ab


In [15]:
def various(notes):
    flag = True
    for i in range(8, len(notes)):
        flag = len(np.unique(notes[i-8:i])) > 2
        if not flag:
            break
    return flag

In [16]:
phrase_len = 60
X = []
y = []
for t in tunes:
    for i in range(len(t) - phrase_len):
        if various(t.iloc[i:i + phrase_len, 1]):
            X.append(t.iloc[i:i + phrase_len, :3])
            y.append(t.iloc[i + phrase_len, :3])
X = np.array(X)
y = np.array(y)

/home/u188113/tmp/ipykernel_2169008/2787662740.py:4: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  flag = len(np.unique(notes[i-8:i])) > 2


In [17]:
X = X.astype(int)
y = y.astype(int)

In [18]:
X.shape

(9558, 60, 3)

In [ ]:
model = Sequential()
model.add(LSTM(512,return_sequences=False, input_shape=(phrase_len, 3)))
model.add(Dropout(0.5))
model.add(Dense(3, activation='relu'))
model.compile(loss='mae', optimizer='adam')

model.fit(X, y, batch_size=256, epochs=100, validation_split=0.2)

Epoch 1/100
30/30 [==============================] - 12s 336ms/step - loss: 54.9533 - val_loss: 49.6262
Epoch 2/100
30/30 [==============================] - 10s 327ms/step - loss: 43.2836 - val_loss: 39.3405
Epoch 3/100
30/30 [==============================] - 10s 328ms/step - loss: 33.1619 - val_loss: 29.3490
Epoch 4/100
30/30 [==============================] - 10s 328ms/step - loss: 23.7926 - val_loss: 21.2349
Epoch 5/100
30/30 [==============================] - 10s 328ms/step - loss: 17.1346 - val_loss: 17.8497
Epoch 6/100
30/30 [==============================] - 10s 327ms/step - loss: 15.2233 - val_loss: 17.6349
Epoch 7/100
30/30 [==============================] - 10s 327ms/step - loss: 15.1306 - val_loss: 17.6088
Epoch 8/100
30/30 [==============================] - 10s 330ms/step - loss: 15.1289 - val_loss: 17.5834
Epoch 9/100
30/30 [==============================] - 10s 330ms/step - loss: 15.0940 - val_loss: 17.5277
Epoch 10/100
30/30 [==============================] - 10s 325ms/

In [ ]:
def tune_generator(model, name='lstm_tune_'):
    for i in range(3):
        start = np.random.randint(0, len(X)-1)
        pattern = X[start]
        prediction_output = []

        for note_index in range(100):
            prediction_input = np.reshape(pattern, (1, len(pattern), 3))
            prediction = model.predict(prediction_input, verbose=0)
            prediction_output.append(prediction.astype(int)[0])
            pattern = np.append(pattern, prediction, axis = 0)
            pattern = pattern[1:len(pattern)]

        notes = pd.DataFrame(prediction_output, columns=['time', 'note', 'velocity'])
        notes['pause'] = 180
        notes_dict = notes.to_dict('records')
        tune_to_midi(notes_dict, midi_name=name + str(i))

In [ ]:
tune_generator(model, name='lstm_mod12_')

In [ ]:
from music21 import midi

def play_midi(filename):
    mf = midi.MidiFile()
    mf.open(filename)
    mf.read()
    mf.close()
    s = midi.translate.midiFileToStream(mf)
    s.show('midi')

In [ ]:
play_midi('lstm_mod12_2.mid')